In [2]:
import sqlite3

- DB: 데이터 베이스; 데이터 저장소
- DBMS: system software for creating and managing database
    - SQL은 DBMS의 한 종류이다
    - DBMS는 server이지만 SQLite는 serverless로 managing이 가능한 pacakge이다

1. python ----> 2. DBMS ----> 3. DB
                  SQLite

1. connection -> client -> server
    * DBMS server와 연결시키는 작업 
2. cursor
   * DBMS에서 실제 작업
3. fetch
   * cursor에서 한 작업을 python을 패치/불러드린다
   * Fetches one/more rows from the resultset
4. commit
   * 실제 DB에 적용  

```
CREATE TABLE tableName(
    COL1 DATATYPE 조건,
    COL2 DATATYPE 조건,
    ...
    );
```

```
# 전체 추가 -> VALUES 뒤에 all columns에 들어갈 데이터
INSERT INTO tablename VALUES (val1, val2, ...)
# 부분 추가 -> 해당 columns에 해당하는 추가할 데이터 
INSERT INTO tablename (COL1, COL2, ...) VALUES (val1, val2, ...)
```

```
cur.execute
Qmark Style:
INSERT INTO ALBUM VALUES (NULL, ?, ?)', (val1, val2)

Named Style:
INSERT INTO ALBUM VALUES (NULL, :name, :no)', {'name':val1, 'no':val2}
```

* PK & FK 정리
    * PK란? 해당 테이블의 row들을 uniquely identify 할 수 있는 숫자/요소
        * 따라서 최소한의 중복만 허용이 가능하다 
    * FK란? 다른 특정 테이블의 row들을 uniquely identify 할 수 있는 숫자/요소
        * 중복 허용이 가능하다 why? 현 테이블의 PK에 따라 FK가 중복이 될 수 있기 때문 

In [3]:
# playlist라는 db를 sqlite3 server를 통해 파이썬과 커넥트 시키는 작업 
conn = sqlite3.connect('playlist.db')

In [4]:
# 연결된 db에서 작업을 하기 위한 마우스포인터라고 생각하자 
cur = conn.cursor()

In [5]:
cur.executescript('''
    DROP TABLE IF EXISTS ARTIST;
    CREATE TABLE ARTIST(
        PK INTEGER PRIMARY KEY,
        NAME TEXT NOT NULL DEFAULT '가수'
    );
''')

In [6]:
cur.executescript('''
    DROP TABLE IF EXISTS ALBUM;
    CREATE TABLE ALBUM(
        PK INTEGER PRIMARY KEY,
        NAME TEXT, 
        FK INTEGER NOT NULL
    );
    
    DROP TABLE IF EXISTS GENRE;
    CREATE TABLE GENRE(
        PK INTEGER PRIMARY KEY,
        NAME TEXT NOT NULL
    );

    DROP TABLE IF EXISTS TRACK;
    CREATE TABLE TRACK(
        PK INTEGER PRIMARY KEY,
        NAME TEXT NOT NULL,
        LENGTH INTEGER DEFAULT 0,
        RATING INTEGER DEFAULT 0,
        COUNT INTEGER DEFAULT 0,
        AFK INTEGER NOT NULL,
        GFK INTEGER NOT NULL
    );
''')
conn.commit()

In [7]:
cur.executemany('''
    INSERT INTO ARTIST VALUES (NULL, ?);
''', [['수지'], ['박효신'], ['아이유']])

In [8]:
cur.executemany('''
    INSERT INTO GENRE VALUES (NULL, ?);
''', [['발라드'], ['R&B'], ['락']])

In [9]:
# ALBUM TABLE을 채우는 함수

# 순서 
# 아티스트 이름에 해당하는 PK를 뽑는다
# 앨범 테이블에 해당 PK와 앨범을 추가시킨다 
# 이때 뽑은 PK는 앨범의 FK가 된다 artist pk = album fk 

def addAlbum(artist, album):
    # 해당 아티스트의 PK 추출 
    cur.execute('SELECT PK FROM ARTIST WHERE NAME LIKE ?', ['%'+artist+'%'])
    # 파이썬에 패치 
    # 바로 cur.fetchone()[0] 하지 않는 이유는:
    # TypeError: 'NoneType' object is not subscriptable
    # 아무것도 없을 경우 NoneType을 읽어들일 수가 없기 때문 
    PK = cur.fetchone()
    
    # PK가 존재한다면 FK로 할당하고 데이터 추가 
    if PK:
        # 왜 인덱싱을 써서 해주냐?
        # 패치는 튜플로 반환하기 때문에 순서대로 반환하다. 
        # 따라서 하나의 column만 반환하더라도 튜플의 형태로 반환하기 때문에 한개의 숫자로 변환 
        FK = PK[0]
        # 데이터 추가 
        # 왜 NULL? bc 첫번째는 pk이니까 
        cur.execute('INSERT INTO ALBUM VALUES (NULL, ?, ?)', (album, FK))
        conn.commit()
    # what's the purpose of the return?
    return PK, artist, album, cur.lastrowid

In [10]:
addAlbum('수지','수지앨범')
addAlbum('박효신','박효신앨범')
addAlbum('아이유','아이유앨범')

((3,), '아이유', '아이유앨범', 3)

In [11]:
# TRACK 테이블을 채우는 함수 

# 순서 
# album 인자에 해당하는 PK 추출 
# album pk = track AFK
# 장르 인자에 해당하는 PK 추출
# genre pk = track GFK
# AFK와 GFK가 동시에 있는 경우 TRACK에 데이터 추가 

def addTrack(album, genre, track, length=0, rating=0, count=0):
    cur.execute('SELECT PK FROM ALBUM WHERE NAME LIKE ?', ['%'+album+'%'])
    # 튜플로 반환 
    PK1 = cur.fetchone()

    cur.execute('SELECT PK FROM GENRE WHERE NAME LIKE ?', ['%'+genre+'%'])
    # 튜플로 반환 
    PK2 = cur.fetchone()

    if PK1 and PK2:
        AFK = PK1[0] 
        GFK = PK2[0]
        cur.execute('''
            INSERT INTO TRACK 
            VALUES (NULL, :name, :length, :rating, :count, :afk, :gfk)
            ''', {'name':track, 'length':length, 'rating':rating, 'count':count, 'afk':AFK, 'gfk':GFK})
        conn.commit()
    return PK1, PK2


In [12]:
addTrack('박효신','R&B', '박효신노래1')
addTrack('박효신','락', '박효신노래2')
addTrack('박효신','락', '박효신노래3')
addTrack('아이유','발라드', '아이유노래1')
addTrack('아이유','락', '아이유노래2')
addTrack('아이유','발라드', '아이유노래3')

((3,), (1,))

In [13]:
# NOTE
#cur.execute('SELECT PK FROM ALBUM WHERE NAME LIKE ?', ('%수지%'))
#cur.fetchone()
# 왜 하나뿐인데도 ,를 쓰냐?
# ,를 쓰지 않으면 튜플이 되지 않기 때문
# 그렇게 되면 문자를 하나씩 iter하게 되는데 이때 ?와 갯숫가 맞지 않기 때문에 에러가 뜬다 

# cur.executemany('SELECT PK FROM ALBUM WHERE NAME LIKE ?', [['%수지%']])
# cur.fetchone()
# ProgrammingError: executemany() can only execute DML statements.
# SELECT DML이 아님.

# iterable 하기만 하면 튜플 리스트 상관이 없다 
# 튜플이랑 리스트를 쓰는 iterable 하게 하기 위해서 

cur.execute('SELECT PK FROM ALBUM WHERE NAME LIKE ?', ('%수지%',))
cur.fetchone()

(1,)

In [14]:
cur.execute('SELECT PK FROM ALBUM WHERE NAME LIKE ?', ['%수지%'])
cur.fetchone()

(1,)

In [15]:
# TRACK NAME을 PK로 바꾼다
# TRACK NAME의 PK를 바꿔줄거야 
cur.execute('''
    UPDATE TRACK
    SET NAME=?
    WHERE PK=?
''', ('아이유노래2', 2))
conn.commit()

In [16]:
# CHECK
cur.execute('SELECT DISTINCT(NAME) FROM ARTIST')
cur.fetchall()

[('수지',), ('박효신',), ('아이유',)]

In [17]:
# CHECK
cur.execute('SELECT * FROM ALBUM')
cur.fetchall()

[(1, '수지앨범', 1), (2, '박효신앨범', 2), (3, '아이유앨범', 3)]

### join

(1)
SELECT ____
FROM ARTIST

(2)
SELECT ___
FROM ARTIST
INNER JOIN ALBUM
ON ALBUM.FK = ARTIST.PK

(3)
SELECT ARTIST>NAME, ALBUM.NAME

In [18]:
# ARTIST - ALBUM
# FROM 기본이 될 것 
# LEFT JOIN 합칠 테이블 
# ON 기본일 될 PK = 합치는 FK
cur.execute('''
    SELECT A.NAME, B.NAME
    FROM ARTIST AS A
    LEFT JOIN ALBUM AS B
    ON A.PK = B.FK
''')
cur.fetchall()

[('수지', '수지앨범'), ('박효신', '박효신앨범'), ('아이유', '아이유앨범')]

In [19]:
# ALBUM - TRACK
cur.execute('''
    SELECT *
    FROM ALBUM AS B
    LEFT JOIN TRACK AS C
    ON B.PK = C.AFK
''')

# 기본이 되는 것은 ALBUM TABLE -> 따라서 album의 요소들은 모두 들어가야 함
# 여기에 track을 가져다 붙힘 by album pk로 
# 그렇게 되면 중복되는 album name이 많아질 수 있음 
cur.fetchall()

[(1, '수지앨범', 1, None, None, None, None, None, None, None),
 (2, '박효신앨범', 2, 1, '박효신노래1', 0, 0, 0, 2, 2),
 (2, '박효신앨범', 2, 3, '박효신노래3', 0, 0, 0, 2, 3),
 (2, '박효신앨범', 2, 2, '아이유노래2', 0, 0, 0, 2, 3),
 (3, '아이유앨범', 3, 4, '아이유노래1', 0, 0, 0, 3, 1),
 (3, '아이유앨범', 3, 5, '아이유노래2', 0, 0, 0, 3, 3),
 (3, '아이유앨범', 3, 6, '아이유노래3', 0, 0, 0, 3, 1)]

In [20]:
cur.execute('''
    SELECT B.NAME, C.NAME
    FROM TRACK AS C
    LEFT JOIN ALBUM AS B
    ON B.PK = C.AFK
''')

# 반대로 기준점이 track이 되어버리면 track의 값들만 모두 있으면 되므로
# track이 없는 수지앨범은 없어짐 
cur.fetchall()

[('박효신앨범', '박효신노래1'),
 ('박효신앨범', '아이유노래2'),
 ('박효신앨범', '박효신노래3'),
 ('아이유앨범', '아이유노래1'),
 ('아이유앨범', '아이유노래2'),
 ('아이유앨범', '아이유노래3')]

In [21]:
# 앨범별 트랙의 갯수 
# album과 track left join
# group by -> 앨범별 트랙의 갯수이기 때문에 앨범을 grouping
# 그 뒤 트랙의 갯수를 세어야하기 때문에 각 앨범마다 몇개의 Name이 있는지 count 

cur.execute('''
    SELECT B.NAME, COUNT(C.NAME)
    FROM ALBUM AS B
    LEFT JOIN TRACK AS C
    ON B.PK = C.AFK
    GROUP BY B.NAME
''')

cur.fetchall()

[('박효신앨범', 3), ('수지앨범', 0), ('아이유앨범', 3)]

In [22]:
# 가수별 앨범 갯수 
cur.execute(''' 
    SELECT A.NAME, COUNT(B.NAME)
    FROM ARTIST AS A
    LEFT JOIN ALBUM AS B
    ON A.PK = B.FK
    GROUP BY A.NAME
''')
cur.fetchall()

[('박효신', 1), ('수지', 1), ('아이유', 1)]

In [23]:
# ARTIST - ALBUM - TRACK
# 하나씩 차근히 조인하는 방법
cur.execute('''
    SELECT A.NAME, B.NAME, C.NAME
    FROM ARTIST AS A
    LEFT JOIN ALBUM AS B
    ON A.PK = B.FK
    LEFT JOIN TRACK AS C
    ON B.PK = C.AFK
''')
cur.fetchall()

[('수지', '수지앨범', None),
 ('박효신', '박효신앨범', '박효신노래1'),
 ('박효신', '박효신앨범', '박효신노래3'),
 ('박효신', '박효신앨범', '아이유노래2'),
 ('아이유', '아이유앨범', '아이유노래1'),
 ('아이유', '아이유앨범', '아이유노래2'),
 ('아이유', '아이유앨범', '아이유노래3')]

In [24]:
# 중첩 
cur.execute('''
    SELECT A.*, B.NAME, B.TNAME
    FROM ARTIST AS A
    LEFT JOIN (
        SELECT ALBUM.FK, ALBUM.NAME AS NAME, TRACK.NAME AS TNAME
        FROM ALBUM 
        LEFT JOIN TRACK
        ON ALBUM.PK = TRACK.AFK
    ) AS B
    ON A.PK = B.FK
''')

# 아래는 album-track을 조인
'''
[(1, '수지앨범', 1, None, None, None, None, None, None, None),
 (2, '박효신앨범', 2, 1, '박효신노래1', 0, 0, 0, 2, 2),
 (2, '박효신앨범', 2, 2, '박효신노래2', 0, 0, 0, 2, 3),
 (2, '박효신앨범', 2, 3, '박효신노래3', 0, 0, 0, 2, 3),
 (3, '아이유앨범', 3, 4, '아이유노래1', 0, 0, 0, 3, 1),
 (3, '아이유앨범', 3, 5, '아이유노래2', 0, 0, 0, 3, 3),
 (3, '아이유앨범', 3, 6, '아이유노래3', 0, 0, 0, 3, 1)]
'''

# album 테이블의 pk와 track의 AFK 맞는 것들끼리 조인 
# 이떄 joined 테이블의 row 갯수가 album 테이블의 갯수보다 작거나 많음
# why? track에서 중복되는 AFK 값을 가지고 있으면 그만큼 row가 더 생생되기 때문
# 위의 해당 테이블의 fk [3번째]와 artist의 pk를 맞춤
# 헷갈리는 부분 예시) 앨범: 김예지, 노래: 김예지1, 김예지2
# 앨범 + 노래: [[김예지, 김예지1], [김예지, 김예지2]]

cur.fetchall()

[(1, '수지', '수지앨범', None),
 (2, '박효신', '박효신앨범', '박효신노래1'),
 (2, '박효신', '박효신앨범', '박효신노래3'),
 (2, '박효신', '박효신앨범', '아이유노래2'),
 (3, '아이유', '아이유앨범', '아이유노래1'),
 (3, '아이유', '아이유앨범', '아이유노래2'),
 (3, '아이유', '아이유앨범', '아이유노래3')]

In [25]:
# where 활용
# 결국 A.PK = B.FK AND B.PK=C.AFK 되는 것들만 조인이 되므로 
# 조인과 같다
# 하지만 None이 있을경우 봔환하지 못함 
cur.execute('''
    SELECT A.NAME, B.NAME, C.NAME
    FROM ARTIST AS A, ALBUM AS B, TRACK AS C
    WHERE A.PK = B.FK AND B.PK=C.AFK   
''')
cur.fetchall()

[('박효신', '박효신앨범', '박효신노래1'),
 ('박효신', '박효신앨범', '아이유노래2'),
 ('박효신', '박효신앨범', '박효신노래3'),
 ('아이유', '아이유앨범', '아이유노래1'),
 ('아이유', '아이유앨범', '아이유노래2'),
 ('아이유', '아이유앨범', '아이유노래3')]

In [26]:
# 앨범별 트랙수
cur.execute('''
    SELECT A.NAME, B.NAME, COUNT(C.NAME)
    FROM ARTIST AS A, ALBUM AS B, TRACK AS C
    WHERE A.PK = B.FK AND B.PK=C.AFK  
    GROUP BY B.NAME
''')
cur.fetchall()

[('박효신', '박효신앨범', 3), ('아이유', '아이유앨범', 3)]

In [27]:
# 가수 - 앨범 - 장르 - 노래
cur.execute('''
    SELECT A.NAME, B.NAME, C.NAME, D.NAME
    FROM ARTIST AS A, ALBUM AS B, GENRE AS C, TRACK AS D
    WHERE A.PK =B.FK AND B.PK = D.AFK AND C.PK=D.GFK
    GROUP BY A.PK, B.PK, C.PK
''')
cur.fetchall()
# 가수를 먼저 묶고, 그 후에 앨범 별로 묶고, 그 후에 장르 별로 묶고, 카운트를 센다 

[('박효신', '박효신앨범', 'R&B', '박효신노래1'),
 ('박효신', '박효신앨범', '락', '아이유노래2'),
 ('아이유', '아이유앨범', '발라드', '아이유노래1'),
 ('아이유', '아이유앨범', '락', '아이유노래2')]

In [28]:
# 가수 > 앨범 > 장르 >  
cur.execute('''
    SELECT A.NAME, B.NAME, C.NAME, D.NAME, COUNT(D.NAME)
    FROM ARTIST AS A, ALBUM AS B, GENRE AS C, TRACK AS D
    WHERE A.PK =B.FK AND B.PK = D.AFK AND C.PK=D.GFK
    GROUP BY A.PK, B.PK, C.PK
''')
# D.NAME에서 박효신노래2만 나오는 이유는 대표되는 것, 그러니까 제일 처음 들어갔던 데이터만 표시가 됨 
cur.fetchall()

[('박효신', '박효신앨범', 'R&B', '박효신노래1', 1),
 ('박효신', '박효신앨범', '락', '아이유노래2', 2),
 ('아이유', '아이유앨범', '발라드', '아이유노래1', 2),
 ('아이유', '아이유앨범', '락', '아이유노래2', 1)]

In [33]:
# view table
# when you want to join the tables by 중첩
# you can make the subset of the table as view table 
cur.executescript('''
    DROP VIEW IF EXISTS ALBUM_TRACK;
    CREATE VIEW ALBUM_TRACK AS
    SELECT TRACK.NAME AS TNAME, ALBUM.NAME AS NAME, ALBUM.FK
    FROM ALBUM 
    LEFT JOIN TRACK
    ON TRACK.AFK = ALBUM.PK 
''')

In [38]:
# 중첩 (ARTIST - VIEW)
cur.execute('''
    SELECT A.NAME, B.NAME, B.TNAME, COUNT(B.TNAME)
    FROM ARTIST AS A
    LEFT JOIN ALBUM_TRACK AS B
    ON A.PK = B.FK
    GROUP BY A.PK
''')
cur.fetchall()

[('수지', '수지앨범', None, 0),
 ('박효신', '박효신앨범', '박효신노래1', 3),
 ('아이유', '아이유앨범', '아이유노래1', 3)]